<a href="https://colab.research.google.com/github/Sai-sakunthala/Assignment1/blob/main/Assignment1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wandb login

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: sai-sakunthala (sai-sakunthala-indian-institute-of-technology-madras) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [1]:
from keras.datasets import fashion_mnist
#import wandb
import numpy as np
import math
import random

In [ ]:
wandb.init(
    # set the wandb project where this run will be logged
    project="Fashion-mnist", name = "Images")

wandb: Currently logged in as: sai-sakunthala (sai-sakunthala-indian-institute-of-technology-madras) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


In [72]:
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
x_train = x_train/255
x_test = x_test/255
class_names = ['Tshirt', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'bag', 'Ankleboot']

one_per_label = {}
for image,label in zip(x_train,y_train):
    if label not in one_per_label:
        one_per_label[label] = image

In [ ]:
for label, image in one_per_label.items():
    wandb.log({"Images": [wandb.Image(image, caption=class_names[label])]})

In [73]:
def input_layer(x):
    x = np.array(x)
    if len(x.shape) == 3:
        x = x.reshape(x.shape[0], -1)
    return x

def sigmoid(a_x):
    a_x = np.clip(a_x, -700, 700)
    h_x = 1 / (1 + np.exp(-a_x))
    return h_x

def der_sigmoid(a_x):
    sig_x = sigmoid(a_x)
    del_sig = sig_x * (1 - sig_x)
    return del_sig

def Relu(a_x):
    h_x = np.clip(np.maximum(0, a_x), 0, 1e4)
    return h_x

def der_Relu(a_x):
    del_Relu = (a_x > 0).astype(float)
    return del_Relu

def tanh(a_x):
    a_x = np.clip(a_x, -700, 700)
    h_x = (np.exp(a_x) - np.exp(-a_x))/(np.exp(a_x) + np.exp(-a_x))
    return h_x

def der_tanh(a_x):
    del_tanh = 1 - ((np.exp(a_x) - np.exp(-a_x))/(np.exp(a_x) + np.exp(-a_x)))**2
    return del_tanh

def softmax(a_x):
    a_x = a_x - np.max(a_x)
    h_x = np.exp(a_x)
    h_x = h_x/np.sum(h_x)
    return h_x

def initialize_weights(num_neurons):
    np.random.seed(42)
    weights = []
    biases = []
    for i in range(len(num_neurons)-1):
        W = np.random.randn(num_neurons[i+1], num_neurons[i])*np.sqrt(1 / num_neurons[i])
        b = np.zeros((1, num_neurons[i+1]))
        weights.append(W)
        biases.append(b)
    return weights, biases

def pre_activation(h_x, W, b):
    a_x = np.dot(W, h_x.T) + b.flatten()
    return a_x

def bce_loss_function(h_x, y):
    h_x = np.clip(h_x, 1e-8, 1.0)
    loss = -np.log(h_x[np.argmax(y)])
    return loss

def mse_loss_function(h_x, y):
    loss = np.mean((h_x - y)**2)
    return loss

def forward_pass(x, y, weights, biases, activation_func, n_hidden, loss_function):
    activations = []
    pre_activations = []
    for i in range(n_hidden+1):
        a_x = pre_activation(x if i == 0 else activations[-1], weights[i], biases[i])
        h_x = softmax(a_x) if i == n_hidden else activation_func(a_x)
        activations.append(h_x)
        pre_activations.append(a_x)
    loss = loss_function(h_x, y)
    return activations, pre_activations, loss

#a,b,_ = forward_pass(np.array([1,1]), np.array([0]), initialize_weights([2,3,3,2])[0], initialize_weights([2,3,3,2])[1], tanh, 2)

def one_hot_encode(y, num_classes):
    return np.eye(num_classes)[y]

def back_propagation(activations, pre_activations, weights, biases, x, y, y_hat, n_hidden, activation_deriv, loss_function):
    del_L_a = {}
    del_L_w = {}
    del_L_b = {}
    del_L_h = {}
    for i in range(n_hidden, -1, -1):
        if i == n_hidden:
            if loss_function == bce_loss_function:
                del_L_a[i] = y_hat - y
        if i == 0:
            del_L_w[i] = np.dot(del_L_a[i][:, np.newaxis], x[np.newaxis, :])
            del_L_b[i] = del_L_a[i]
            break
        else:
            del_L_w[i] = np.dot(del_L_a[i][:, np.newaxis], activations[i-1][np.newaxis, :])
        del_L_b[i] = del_L_a[i]
        del_L_h[i-1] = np.matmul(weights[i].T, del_L_a[i])
        del_L_a[i-1] = del_L_h[i-1]*activation_deriv(pre_activations[i-1])
    return del_L_w, del_L_b

def gradient_descent(dw, db, weights, biases, learning_rate):
    '''
    for key in dw:
        dw[key] = np.clip(dw[key], -5, 5)'''
    for i in range(len(weights)):
        weights[i] -= learning_rate*dw[i]
        biases[i] -= learning_rate*db[i]
    return weights, biases

def momentum_gradient(dw, db, weights, biases, learning_rate, beta, prev_u_w, prev_u_b):
    u_w = {}
    u_b = {}
    for i in range(len(weights)):
        if prev_u_w == {} and prev_u_b == {}:
            u_w[i] = learning_rate*dw[i]
            u_b[i] = learning_rate*db[i]
        else:
            u_w[i] = beta*prev_u_w[i] + learning_rate*dw[i]
            u_b[i] = beta*prev_u_b[i] + learning_rate*db[i]
        weights[i] = weights[i] - u_w[i]
        biases[i] = biases[i] - u_b[i]
    prev_u_w = u_w
    prev_u_b = u_b
    return weights, biases, prev_u_w, prev_u_b

def nestrov_gradient(dw, db, weights, biases, learning_rate, beta, prev_u_w, prev_u_b):
    u_w = {}
    u_b = {}
    for i in range(len(weights)):
        if prev_u_w == {} and prev_u_b == {}:
            u_w[i] = learning_rate*dw[i]
            u_b[i] = learning_rate*db[i]
        else:
            u_w[i] = beta*prev_u_w[i] + learning_rate*dw[i]
            u_b[i] = beta*prev_u_b[i] + learning_rate*db[i]
        weights[i] = weights[i] - u_w[i]
        biases[i] = biases[i] - u_b[i]
    prev_u_w = u_w
    prev_u_b = u_b
    return weights, biases, prev_u_w, prev_u_b

def validation(x_val, y_val, weights, biases, activation_func, n_hidden, loss_function):
    val_loss_final = 0
    y_pred_val = []
    y_val_j = []
    for j in range(0, len(x_val)):
        x_val_each = x_val[j]
        y_val_each = y_val[j]
        activ, _,val_loss = forward_pass(x_val_each, y_val_each, weights, biases, activation_func, n_hidden, loss_function)
        a_1 = activ[-1]
        y_pred_val.append(np.argmax(a_1))
        y_val_j.append(np.argmax(y_val[j]))
        val_loss_final = val_loss_final + val_loss
    accuracy = np.mean(np.array(y_pred_val) == np.array(y_val_j))
    print("Validation Loss: ", val_loss_final)
    print("Validation Accuracy: ", accuracy)

def Neuralnet(x_train, y_train, n_hidden, n_neurons_hidden, epochs, batch_size, activation, optimization, learning_rate, weight_decay, loss_function):
    x_train = input_layer(x_train)
    classes = len(np.unique(y_train))
    y_train = one_hot_encode(y_train, classes)
    features = x_train.shape[1]
    split_index = int(0.9 * x_train.shape[0])
    indices = np.random.permutation(len(x_train))
    x_train, y_train = x_train[indices], y_train[indices]
    x_train_final, x_val = x_train[:split_index], x_train[split_index:]
    y_train_final, y_val = y_train[:split_index], y_train[split_index:]
    num_neurons = [features] + [n_neurons_hidden]*(n_hidden) + [classes]
    weights, biases = initialize_weights(num_neurons)
    activation_func = {"sigmoid": sigmoid, "tanh": tanh, "relu": Relu}[activation]
    activation_deriv = {"sigmoid": der_sigmoid, "tanh": der_tanh, "relu": der_Relu}[activation]
    optimization_func = {"momentum": momentum_gradient, "sgd": gradient_descent, "nestrov": nestrov_gradient}[optimization]
    loss_function = {"bce": bce_loss_function, 'mse': mse_loss_function}[loss_function]

    if optimization_func == gradient_descent:
        for epoch in range(epochs):
            epoch_loss = 0
            for i in range(0, len(x_train), batch_size):
                x_batch = x_train[i:i + batch_size]
                y_batch = y_train[i:i + batch_size]
                dw = {}
                db = {}
                for x,y in zip(x_batch,y_batch):
                    activations, pre_activations, loss = forward_pass(x, y, weights, biases, activation_func, n_hidden, loss_function)
                    epoch_loss += loss
                    del_L_w, del_L_b = back_propagation(activations, pre_activations, weights, biases, x, y, activations[-1], n_hidden, activation_deriv, loss_function)
                    for key,value in del_L_w.items():
                        if key not in dw:
                            dw[key] = value
                        else:
                            dw[key] = dw[key] + value
                    for key,value in del_L_b.items():
                        if key not in db:
                            db[key] = value
                        else:
                            db[key] = db[key] + value
                for key in dw:
                    dw[key] /= batch_size
                    db[key] /= batch_size
                weights, biases = optimization_func(dw, db, weights, biases, learning_rate)
            epoch_loss /= len(x_train)
            print('epoch:', epoch, 'training loss:', epoch_loss)
            validation(x_val, y_val, weights, biases, activation_func, n_hidden, loss_function)
            print('')

    elif optimization_func == momentum_gradient:
        prev_u_w = {}
        prev_u_b = {}
        for epoch in range(epochs):
            for i in range(0, len(x_train), batch_size):
                x_batch = x_train[i:i + batch_size]
                y_batch = y_train[i:i + batch_size]
                dw = {}
                db = {}
                for x,y in zip(x_batch,y_batch):
                    activations, pre_activations, loss = forward_pass(x, y, weights, biases, activation_func, n_hidden, loss_function)
                    del_L_w, del_L_b = back_propagation(activations, pre_activations, weights, biases, x, y, activations[-1], n_hidden, activation_deriv, loss_function)
                    for key,value in del_L_w.items():
                        if key not in dw:
                            dw[key] = value
                        else:
                            dw[key] = dw[key] + value
                    for key,value in del_L_b.items():
                        if key not in db:
                            db[key] = value
                        else:
                            db[key] = db[key] + value
                weights, biases, prev_u_w, prev_u_b = optimization_func(dw, db, weights, biases, learning_rate, weight_decay, prev_u_w, prev_u_b)
    elif optimization_func == nestrov_gradient:
        prev_u_w = {}
        prev_u_b = {}
        for epoch in range(epochs):
            for i in range(0, len(x_train), batch_size):
                x_batch = x_train[i:i + batch_size]
                y_batch = y_train[i:i + batch_size]
                dw = {}
                db = {}
                for x,y in zip(x_batch,y_batch):
                    activations, pre_activations, loss = forward_pass(x, y, weights, biases, activation_func, n_hidden, loss_function)
                    if i == 0:
                        del_L_w, del_L_b = back_propagation(activations, pre_activations, weights, biases, x, y, activations[-1], n_hidden, activation_deriv, loss_function)
                    else:
                        look_ahead_weights = {key: weights[key] - weight_decay*prev_u_w[key] for key in range(len(weights))}
                        look_ahead_biases = {key: biases[key] - weight_decay*prev_u_b[key] for key in range(len(weights))}
                        del_L_w, del_L_b = back_propagation(activations, pre_activations, look_ahead_weights, look_ahead_biases, y, activations[-1], n_hidden, activation_deriv, loss_function)
                    for key,value in del_L_w.items():
                        if key not in dw:
                            dw[key] = value
                        else:
                            dw[key] = dw[key] + value
                    for key,value in del_L_b.items():
                        if key not in db:
                            db[key] = value
                        else:
                            db[key] = db[key] + value
                weights, biases, prev_u_w, prev_u_b = optimization_func(dw, db, weights, biases, learning_rate, weight_decay, prev_u_w, prev_u_b)

Neuralnet(x_train, y_train, 3, 64, 15, 128, 'sigmoid', 'sgd', 0.1, 0.9, 'bce')

#Neuralnet(np.array([[1,1],[2,2],[3,3],[0,0]]), np.array([0,1,1,0]), 3, 100, 5000, 2, 'sigmoid', 'sgd', 0.1, 0.9, 'bce')

epoch: 0 training loss: 2.282242149931839
Validation Loss:  13281.283955021187
Validation Accuracy:  0.31683333333333336

epoch: 1 training loss: 1.788281852175081
Validation Loss:  8563.132947909868
Validation Accuracy:  0.5183333333333333

epoch: 2 training loss: 1.2445182555443361
Validation Loss:  6537.825069818697
Validation Accuracy:  0.6311666666666667

epoch: 3 training loss: 0.9907647841725996
Validation Loss:  5320.3165620551845
Validation Accuracy:  0.6736666666666666

epoch: 4 training loss: 0.8375897208844385
Validation Loss:  4677.95313778669
Validation Accuracy:  0.7081666666666667

epoch: 5 training loss: 0.752554625982571
Validation Loss:  4274.072530926773
Validation Accuracy:  0.7336666666666667

epoch: 6 training loss: 0.6939109031448687
Validation Loss:  3962.1114374670906
Validation Accuracy:  0.7596666666666667

epoch: 7 training loss: 0.6477139371837154
Validation Loss:  3710.0032629744355
Validation Accuracy:  0.7756666666666666

epoch: 8 training loss: 0.61060